## Note - This example code uses a SQL Server instance running on a docker Linux container.  If you have created a container using the BuildContainer notebook, use these scripts as is.  If you are running on a native instance, modify the scripts according the comments.  

## Step 1 - Connect to your SQL Server instance using Azure Data Studio.

In [16]:
/* If you created a SQL container in the previous step, you can use this script as is.
   If you are using a different instance, change the FILENAME paths to be valid.
   Change the database and object names if you must.  */
USE master
GO
DROP DATABASE IF EXISTS BlackSabbath
GO
CREATE DATABASE BlackSabbath
ON PRIMARY
(NAME = N'BlackSabbath_primary', FILENAME = '/var/opt/mssql/data/BlackSabbath.mdf', SIZE = 2Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
LOG ON 
(NAME = N'BlackSabbath_Log', FILENAME = '/var/opt/mssql/data/BlackSabbath_log.ldf', SIZE = 5Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
GO

ALTER DATABASE BlackSabbath
ADD FILEGROUP PVS_FG;
GO
ALTER DATABASE BlackSabbath
ADD FILE
(
    NAME = BlackSabbath_pvs,
    FILENAME = '/var/opt/mssql/data/BlackSabbath_pvs.ndf',
    SIZE = 1GB,
    FILEGROWTH = 1MB
)
TO FILEGROUP PVS_FG;
GO

ALTER DATABASE BlackSabbath SET RECOVERY SIMPLE
GO
USE BlackSabbath
GO
DROP TABLE IF EXISTS NeverSayDie
GO
CREATE TABLE NeverSayDie (rowid int IDENTITY(1,1) primary key clustered, rowtext char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
DECLARE @x int
SET @x = 0
WHILE (@x < 11111)
BEGIN
	INSERT INTO NeverSayDie VALUES 
    ('Never Say Die'),
    ('Johnny Blade'),
    ('Junior''s Eyes'),
    ('Hard Road'),
    ('Shock Wave'),
    ('Air Dance'),
    ('Over to You'),
    ('Breakout (Instrumental)'),
    ('Swinging the Chain')
	SET @x = @x + 1
END
COMMIT TRAN
GO
SET NOCOUNT OFF

USE master;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'BlackSabbath';



Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:21.940

name,is_accelerated_database_recovery_on
BlackSabbath,0


## Step 2 - Delete all rows for the table in an open transaction.

In [17]:
USE master
GO
ALTER DATABASE BlackSabbath SET ACCELERATED_DATABASE_RECOVERY = OFF
GO
USE BlackSabbath
GO
BEGIN TRAN
DELETE from NeverSayDie
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(99999 rows affected)

Total execution time: 00:00:44.754

## Step 3 - Check transaction log usage.
### Because there is an open transaction, the log file will contain space that cannot be reused.  After checking log usage, run a CHECKPOINT and see if log space is freed.

In [18]:
USE BlackSabbath;

SELECT * FROM sys.dm_db_log_space_usage

CHECKPOINT;

SELECT * FROM sys.dm_db_log_space_usage

GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.316

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
7,5368700928,4224151552,78.681076,988966912


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
7,5368700928,4224204800,78.68207,1000251392


## Step 4 - How long does it take to rollback the deletes?
### Rollback the transaction and check the duration.

In [19]:
USE BlackSabbath;

ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:46.042

## Step 5 - Enable Accelerated Database Recovery

In [20]:
USE master
GO
ALTER DATABASE BlackSabbath SET ACCELERATED_DATABASE_RECOVERY = ON
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.069

## Step 6 - Delete all of the rows again within a transaction

In [21]:
USE BlackSabbath
GO

BEGIN TRAN
DELETE from NeverSayDie;

GO

Commands completed successfully.

(99999 rows affected)

Total execution time: 00:00:18.881

## Step 7 - Check log space used before and after a CHECKPOINT
### Notice log space usage is low even before the CHECKPOINT

In [22]:
USE BlackSabbath;

SELECT d.name, d.is_accelerated_database_recovery_on, p.persistent_version_store_size_kb 
FROM sys.databases d
INNER JOIN sys.dm_tran_persistent_version_store_stats p
ON p.database_id = d.database_id
WHERE is_accelerated_database_recovery_on = 1;

SELECT * FROM sys.dm_db_log_space_usage
GO

CHECKPOINT;

SELECT * FROM sys.dm_db_log_space_usage
GO


(2 rows affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.112

name,is_accelerated_database_recovery_on,persistent_version_store_size_kb
whitesox,1,0
BlackSabbath,1,0


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
7,5368700928,385654784,7.183391,106475520


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
7,5368700928,385671168,7.1836963,77824


## Step 8 - How fast is a ROLLBACK?
### Is ROLLBACK any faster with ADR enabled?

In [23]:
USE BlackSabbath;

ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:00.003